<a href="https://colab.research.google.com/github/arsh1207/BigDataProject/blob/master/User_based_CF_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import dask.array as da
from numpy import cov
from numpy import std
from sklearn.metrics.pairwise import cosine_similarity
import random
import math

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark

spark-3.0.0-preview2-bin-hadoop2.7/
spark-3.0.0-preview2-bin-hadoop2.7/data/
spark-3.0.0-preview2-bin-hadoop2.7/data/streaming/
spark-3.0.0-preview2-bin-hadoop2.7/data/streaming/AFINN-111.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_binary_classification_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_kmeans_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_multiclass_classification_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_lda_libsvm_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/iris_libsvm.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/pagerank_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_linear_regression_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/pic_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/test.data
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/sample_movielens_rati

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# User User based Collaborative filtering

In [0]:
def init_spark():
    spark = SparkSession \
        .builder \
        .config("spark.driver.memory", "15g") \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [0]:
spark = init_spark()

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [21]:
!ls "/content/drive/My Drive/Colab Notebooks"

 anime.csv
 Bike_Sharing.ipynb
 Classifier_Interpretability_Project.ipynb
'Copy of Part1_TensorFlow.ipynb'
'Copy of Part2_Music_Generation.ipynb'
 rating.csv
 Untitled0.ipynb
 User_based_CF.ipynb


In [0]:
# Reading Data from anime.csv
df2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/anime.csv')

In [0]:
# Redaing from rating.csv and transforming data
df = spark.read.text("/content/drive/My Drive/Colab Notebooks/rating.csv").rdd
data = df.map(lambda x: x[0].split(','))
data2 = data.groupBy(lambda x : x[0])

In [0]:
data3 = data2.collect()

In [0]:
# Preparing data : a dictionary utility = {users: {anime : ratings}}
utility = {}
all_animes = []
all_users = []
for user in range(len(data3)):
    if data3[user][0] == 'user_id':
        continue
    utility[int(data3[user][0])] = {}
    #if int(data3[user][0]) not in all_users:
    #    all_users.append(int(i[0]))
    for i in data3[user][1]:        
        if int(i[1]) not in all_animes:
            all_animes.append(int(i[1]))
        if int(i[2]) == -1:
            continue            
        utility[int(i[0])][int(i[1])] = int(i[2])

In [13]:
utility

{722: {30: 8,
  32: 8,
  120: 8,
  121: 8,
  199: 9,
  430: 8,
  431: 9,
  523: 9,
  853: 9,
  908: 8,
  934: 8,
  1535: 8,
  1689: 9,
  1723: 8,
  2025: 9,
  2167: 9,
  2236: 9,
  2593: 8,
  2759: 9,
  2890: 9,
  2966: 8,
  3092: 8,
  3457: 7,
  3470: 9,
  3782: 8,
  3783: 8,
  3784: 9,
  4059: 8,
  4081: 9,
  4181: 9,
  4182: 8,
  4224: 9,
  4280: 8,
  4282: 8,
  4814: 8,
  4898: 8,
  5114: 10,
  5204: 8,
  5205: 8,
  5630: 8,
  5680: 8,
  5681: 9,
  6045: 9,
  6127: 7,
  6351: 8,
  6372: 8,
  6421: 9,
  6547: 9,
  6553: 7,
  6573: 8,
  6637: 8,
  6682: 8,
  6707: 9,
  6746: 8,
  6862: 8,
  7054: 9,
  7674: 8,
  7791: 9,
  8063: 8,
  8074: 7,
  8408: 8,
  8630: 8,
  9135: 8,
  9253: 8,
  9617: 9,
  9656: 9,
  9734: 8,
  9756: 9,
  9919: 10,
  9926: 9,
  9989: 9,
  10029: 9,
  10161: 8,
  10278: 8,
  10321: 8,
  10408: 9,
  10604: 7,
  10620: 9,
  10800: 8,
  10863: 8,
  11111: 8,
  11123: 9,
  11553: 8,
  11633: 8,
  11737: 9,
  11757: 7,
  11771: 10,
  11843: 8,
  12189: 9,
  12355:

In [0]:
all_users = list(utility.keys())

In [0]:
del data3

In [0]:
import gc
gc.collect()

80

In [0]:
def sim2(matrix, user1, user2):
    #This method returns the similarity between two users
    #It uses Pearson Correlation Coefficient for similarity calculation
    matched = []
    for anime in matrix[user1].keys():
        if anime in matrix[user2].keys():
            matched.append(anime) 
    x = []
    y = []    
    for anime in matched:
        x.append(matrix[user1][anime])
        y.append(matrix[user2][anime])        

    f = (lambda i, j: i - j)
    x = f(x, np.mean(x))
    y = f(y, np.mean(y))
    # Pearson Correlation Coefficient formula below
    numerator = (lambda x, y : x*y)
    denominator = (lambda x: x*x)
    d = math.sqrt(sum(denominator(x))) * math.sqrt(sum(denominator(y)))
    if d == 0:
        return 0
    similarity = sum(numerator(x, y))/ d
    return similarity

In [0]:
def most_similar_user(matrix, user, n=500):
    # This method returna a list of n most similar users
    similarity = {}
    for p in matrix.keys():
        similarity[p] = sim2(matrix, user, p)
    sorted_list = sorted(similarity.items(), key = lambda kv:(kv[1], kv[0]))
    return sorted_list[-n:]     

In [0]:
def recommend(matrix, user):
    # this method returns the predictions wrt most similar users
    similar_users = most_similar_user(matrix, user)
    predictions = {}
    for anime in all_animes:
        num = 0
        den = 0
        for user_ in similar_users:
            if anime in matrix[user_[0]].keys():
                num += matrix[user_[0]][anime] * user_[1]
                den += user_[1]
        if den == 0:
            continue
        #print(matrix[user[0]])
        #if anime in matrix[user[0]].keys():
        predictions[anime] = float(num/den)
    return predictions

In [0]:
# Creating Test data of randomly selecting 50 users
test= int(0.05 * len(all_users))
test_data_user = []
for _ in range(50):
    random_user = random.randint(1, len(all_users))
    if all_users[random_user] not in test_data_user:
        test_data_user.append(all_users[random_user])

In [0]:
# Creating test data of ramdomly selecting 30% of the total animes list
test_animes = int(0.3 * len(all_animes))
test_data_animes = []
for _ in range(test_animes):
    random_anime = random.randint(1, len(all_animes))
    if all_animes[random_anime] not in test_data_animes:
        test_data_animes.append(all_users[random_anime])

In [0]:
# making a deep copy of the data 
import copy
training_data = copy.deepcopy(utility)

In [0]:
#Creating training data by deleting the test data from the original data
for user in test_data_user:
    for anime in test_data_animes:
        if anime in training_data[user].keys():        
            del training_data[user][anime]                        

In [0]:
def start():
    predicted = {}
    for user in test_data_user:
        predicted[user] = recommend(training_data, user)
        print(user)
    return predicted

In [63]:
predicted1 = start()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


67748
65659
51644
61196
6201
28697
62818
51042
9727
44058
25878
2029
5897
8921
32657
70750
31461
62316
43756
34870
12697
51812
34802
60325
47140
65640
12233
62644
62319
10282
21022
70060
55748
48533
12581
35971
40629
10457
35554
72520
25475
39907
51140
37903
61457
51133
2236
7820
67670
30751


In [0]:
def RMSE(actual, predicted):
    error = 0
    n = 0
    for user in test_data_user:
        for anime in test_data_animes:
            if anime in predicted[user].keys() and anime in actual[user].keys():
                error += (actual[user][anime] - predicted[user][anime])**2
                n += 1
    return math.sqrt(error/n)

In [65]:
RMSE(utility, predicted1)

2.0913927814953595

## Applying the common practice 

In [0]:
avg_rating_of_user = {}
global_avg = 0
counter = 0
for user in all_users:
    count = 0
    summ = 0
    for anime in utility[user].keys():
        summ += utility[user][anime]
        count += 1
        global_avg += utility[user][anime]
        counter += 1
    if count != 0:
        avg_rating_of_user[user] = summ/count
    else:
        avg_rating_of_user[user] = 0
global_avg = global_avg/counter

In [27]:
global_avg

7.808496893124035

In [0]:
avg_rating_of_animes = {}
for anime in all_animes:
    count = 0
    summ = 0
    for user in all_users:
        if anime in utility[user].keys():
            summ += utility[user][anime]
            count += 1
    if count != 0:
        avg_rating_of_animes[anime] = summ/count
    else:
        avg_rating_of_animes[anime] = 0
    

In [0]:
def recommend_with_regularization(matrix, user):
    # this method returns the predictions wrt most similar users
    similar_users = most_similar_user(matrix, user)
    predictions = {}
    for anime in all_animes:
        num = 0
        den = 0
        for user_ in similar_users:
            if anime in matrix[user_[0]].keys():
                b_xj = global_avg + (avg_rating_of_user[user_[0]] - global_avg) + (avg_rating_of_animes[anime]- global_avg)
                num += (matrix[user_[0]][anime]- b_xj) * user_[1]
                den += user_[1]
        if den == 0:
            continue
        # if anime in matrix[user[0]].keys():
        b_xi = global_avg + (avg_rating_of_user[user] - global_avg) + (avg_rating_of_animes[anime] - global_avg)
        if b_xi + float(num/den) > 10:
            predictions[anime] = 10
        else:
            predictions[anime] = b_xi + float(num/den)
    return predictions

In [0]:
def start2():
    predicted = {}
    for user in test_data_user:
        predicted[user] = recommend_with_regularization(training_data, user)
        print(user)
    return predicted

In [0]:
recommend_with_regularization(training_data, 3)

In [82]:
predicted = start2()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


67748
65659
51644
61196
6201
28697
62818
51042
9727
44058
25878
2029
5897
8921
32657
70750
31461
62316
43756
34870
12697
51812
34802
60325
47140
65640
12233
62644
62319
10282
21022
70060
55748
48533
12581
35971
40629
10457
35554
72520
25475
39907
51140
37903
61457
51133
2236
7820
67670
30751


In [83]:
RMSE(utility, predicted)

1.5373710265944023

In [68]:
print(utility[51133])

{59: 10, 68: 10, 93: 10, 94: 10, 177: 10, 210: 9, 232: 10, 242: 9, 251: 9, 371: 10, 372: 8, 431: 10, 455: 9, 476: 10, 486: 7, 907: 8, 969: 10, 1535: 10, 1575: 10, 1579: 10, 1698: 10, 2116: 10, 2251: 10, 2904: 10, 2952: 10, 3392: 10, 3457: 10, 3613: 9, 4477: 10, 4752: 10, 5300: 10, 5678: 10, 7311: 7, 8888: 8, 9253: 10, 9919: 7, 10162: 8, 10379: 10, 11111: 7, 11665: 9, 12413: 9, 16498: 10, 19815: 8, 20507: 9}


In [69]:
print(predicted[51133])

{30: 9.65527621128264, 32: 10.305575999389735, 120: 8.437252348581989, 121: 9.774637900010541, 199: 10.271875798030598, 430: 9.546552087729474, 431: 10.47357649168118, 523: 9.779881829851751, 853: 9.407338408490366, 908: 9.39021343162333, 934: 9.543861011231687, 1535: 10.535675612335666, 1689: 9.598771626635164, 1723: 8.571582033833906, 2025: 9.550555288422458, 2167: 9.640351485326553, 2236: 10.054226339324297, 2593: 8.47357320109363, 2759: 9.88045565026162, 2890: 9.216097506233174, 2966: 9.389911570282827, 3092: 9.458823376289027, 3457: 10.474051211728602, 3470: 8.940808759409268, 3782: 8.716288191560285, 3783: 8.550496396647171, 3784: 10.363341711751739, 4059: 9.635537570946617, 4081: 10.12156102410471, 4181: 10.370902505824883, 4182: 9.117180994332879, 4224: 9.953969394914498, 4280: 8.217163063313839, 4282: 9.526860787143972, 4814: 9.186897281845757, 4898: 9.501997445449044, 5114: 10.490360565407071, 5204: 7.393667787473097, 5205: 9.097799562654178, 5630: 9.249530359492505, 5680: 9.

In [70]:
print(predicted1[51133])

{30: 8.451612903225808, 32: 9.066666666666668, 120: 7.425, 121: 8.709923664122137, 199: 9.21505376344086, 430: 8.394736842105264, 431: 9.461538461538462, 523: 8.705882352941176, 853: 8.441176470588236, 908: 7.874999999999999, 934: 8.295454545454545, 1535: 9.58029197080292, 1689: 8.535714285714286, 1723: 7.277777777777778, 2025: 8.590163934426231, 2167: 8.513274336283185, 2236: 9.0, 2593: 7.555555555555555, 2759: 8.67741935483871, 2890: 8.0, 2966: 8.275862068965518, 3092: 8.444444444444445, 3457: 9.666666666666668, 3470: 8.0, 3782: 7.714285714285714, 3783: 7.5, 3784: 9.172413793103448, 4059: 8.314285714285715, 4081: 9.333333333333334, 4181: 9.16842105263158, 4182: 8.133333333333333, 4224: 8.806451612903226, 4280: 7.166666666666667, 4282: 8.4, 4814: 8.083333333333334, 4898: 8.46875, 5114: 9.5, 5204: 6.666666666666667, 5205: 8.2, 5630: 8.428571428571429, 5680: 8.325, 5681: 8.636363636363637, 6045: 8.615384615384615, 6127: 6.444444444444444, 6351: 7.972222222222222, 6372: 8.461538461538462

In [0]:
def recommend_animes(for_user, n = 10):
    predictions = recommend(training_data, for_user)
    sorted_list = sorted(predictions.items(), key = lambda kv:(kv[1], kv[0]))
    for anime in sorted_list[-n:]:
        values = df2[['anime_id','name', 'genre', 'type']].loc[df2['anime_id']==anime[0]]
        print(str(values['anime_id']) + "," + str(values['name']) + "," + str(values['genre']) + "," + str(values['type']) + "\n")
    

In [77]:
recommend_animes(3)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


113    30503
Name: anime_id, dtype: int64,113    Noragami Aragoto
Name: name, dtype: object,113    Action, Adventure, Shounen, Supernatural
Name: genre, dtype: object,113    TV
Name: type, dtype: object

1817    30991
Name: anime_id, dtype: int64,1817    Himouto! Umaru-chan: Umaru-chan Mou Ikkai!
Name: name, dtype: object,1817    Comedy, School, Seinen, Slice of Life
Name: genre, dtype: object,1817    OVA
Name: type, dtype: object

4954    31121
Name: anime_id, dtype: int64,4954    Monster Musume no Iru Nichijou: Hobo Mainichi ...
Name: name, dtype: object,4954    Comedy, Slice of Life
Name: genre, dtype: object,4954    ONA
Name: type, dtype: object

369    31173
Name: anime_id, dtype: int64,369    Akagami no Shirayuki-hime 2nd Season
Name: name, dtype: object,369    Drama, Fantasy, Romance, Shoujo
Name: genre, dtype: object,369    TV
Name: type, dtype: object

3375    31234
Name: anime_id, dtype: int64,3375    Himouto! Umaru-chanS
Name: name, dtype: object,3375    Comedy, Seinen, Slic

In [0]:
def recommend_animes_regularized(for_user, n = 10):
    predictions = recommend_with_regularization(training_data, for_user)
    sorted_list = sorted(predictions.items(), key = lambda kv:(kv[1], kv[0]))
    for anime in sorted_list[-n:]:
        values = df2[['anime_id','name', 'genre', 'type']].loc[df2['anime_id']==anime[0]]
        print(str(values['anime_id']) + "," + str(values['name']) + "," + str(values['genre']) + "," + str(values['type']) + "\n")
    

In [79]:
recommend_animes_regularized(3)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1156    1488
Name: anime_id, dtype: int64,1156    Area 88
Name: name, dtype: object,1156    Action, Adventure, Drama, Military, Romance
Name: genre, dtype: object,1156    OVA
Name: type, dtype: object

1546    1577
Name: anime_id, dtype: int64,1546    Taiho Shichau zo
Name: name, dtype: object,1546    Action, Comedy, Police, Seinen
Name: genre, dtype: object,1546    OVA
Name: type, dtype: object

2172    1901
Name: anime_id, dtype: int64,2172    11-nin Iru!
Name: name, dtype: object,2172    Action, Adventure, Drama, Mystery, Romance, Sc...
Name: genre, dtype: object,2172    Movie
Name: type, dtype: object

3259    1951
Name: anime_id, dtype: int64,3259    Manie-Manie: Meikyuu Monogatari
Name: name, dtype: object,3259    Adventure, Fantasy, Horror, Sci-Fi, Supernatural
Name: genre, dtype: object,3259    Movie
Name: type, dtype: object

1835    2182
Name: anime_id, dtype: int64,1835    Robot Carnival
Name: name, dtype: object,1835    Fantasy, Sci-Fi
Name: genre, dtype: object,1835    OVA